# **Optical Flow**

## **Start up Code**

**Imports**

In [12]:
import numpy as np
import matplotlib.pyplot as plt

import cv2

## **1. Theory**

I think it'll be more effective to learn about this topic through a video rather than reading about it. You might want to watch a few video on the topic:
- *[Overview of Optical Flow](https://youtu.be/lnXFcmLB7sM?si=DP2z33qdBUr-ta25)*
- *[Motion Field and Optical Flow](https://www.youtube.com/watch?v=fLzhaY90ym4&list=PL2zRqk16wsdoYzrWStffqBAoUY8XdvatV&index=3)*
- *[Optical Flow Constraint Equation](https://youtu.be/IjPLZ3hjU1A?si=ahbaHejQrEFngV7s)*
- *[Lucas-Kanade Method](https://youtu.be/6wMoHgpVUn8?si=7lqpVqi5ElOj-Chm)*

## **2. Implementation**

### **2.1 Example 1: Single Object Tracking w/ Lucas-Kanade**

In this example, we'll demonstrate how to track a single object moving in a video.

Before proceeding, let's discuss what exactly we're doing because it can (will) get a bit messy. Our broader goal is to implement optical flow tracking on a video. Optical flow in computer vision enables us to track the motion of objects in a sequence of images (or frames) over time. It calculates the displacement of certain points (`prevPts` and `nextPts`) between consecutive frames (`prevImg` and `nextImg`), allowing us to estimate the movement patterns within the video.

We'll begin by initializing a video capture object (`cap`) to read frames from a video file.

The function `cv2.VideoCapture()` enables users to read in a video or specify the camera source.
- To read a video, simply provide the file path.
- To read from a camera source, pass the index of the input source. To open the default camera (webcam), pass 0.

In [4]:
# Initialize a VideoCapture object to read frames from the specified video file
cap = cv2.VideoCapture('./data/videos/carchase.mp4')

To proceed, we'll make use of the Lucas-Kanade algorithm by utilizing the calcOpticalFlowPyrLK function provided by OpenCV. This function allows us to estimate the motion of keypoints between two consecutive frames in a video sequence.

`nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts, winSize = , maxLevel = )`
- **prevImg**: The previous frame in the video sequence.
- **nextImg**: The next frame in the video sequence.
- **prevPts**: The vector of 2D points for which the flow needs to be found
- **winSize**: Size of the averaging window for optical flow. It determines the size of the neighborhood considered for calculating optical flow.
- **maxLevel**: Specifies the 0-based maximal pyramid level number. This parameter controls the maximum level of pyramid to use for the algorithm

The return values are as follows:
- **nextPts**: Output vector of 2D points containing the calculated new positions of the input points in the next image.
- **status**: Output status vector. If '1', then 'nextPts[i]' is found. Otherwise, 'nextPts[i]' is not found.
- **err**: Output vector that will contain the difference between patches around the orignal and moved points. This is an optional output and can be omitted if not needed.

We'll go ahead and initialize `prevPts`. This array is used to track the movement of points in subsequent frames.

In [5]:
prevPts = np.array((200, 106)).reshape(-1, 1, 2).astype(np.float32)

We'll now set the window size for the Lucas-kanade optical flow algorithm. Again, this determines the size of the window for tracking points.

In [6]:
winSize = (21, 21)

Moving forward, we can extract individual frames from a video by using the `read()` method on the `VideoCapture` object.

`(ret, frame) = cv2.videoCapture.read()`
- **ret**: A boolean value indicating whether a frame was successfully read.
- **frame**: If 'ret' is 'True', 'frame' contains the next frame read from the video source as a NumPy array in BGR format.

After extracting the frame, we'll proceed to convert it to grayscale. Grayscale images are commonly used for optical flow calculation as they simplify processing.

In [7]:
# Read a frame from the video capture object
ret, frame = cap.read()

# Convert the frame to grayscale using the cvtColor function
prevImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

We can now combine our knowledge to create a loop for processing the frames in the video as follows:

In [8]:
# Loop until the video capture object is open
while cap.isOpened():
    # Read a frame from the video capture object
    ret, frame = cap.read()

    # Convert the frame to grayscale
    nextImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Lucas-Kanade method
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, None, winSize=winSize, maxLevel=3)

    # Filter out points with a valid status
    prevPts = prevPts[status == 1]
    nextPts = nextPts[status == 1]

    # Draw optical flow vectors on the frame
    for i, (old_point, new_point) in enumerate(zip(prevPts, nextPts)):
        frame = cv2.rectangle(
            frame, 
            new_point.astype(int) - winSize[0]//2,  # Draw rectangle around new point
            new_point.astype(int) + winSize[0]//2,  # Draw rectangle around old point
            [0, 0, 255],  # Color of the rectangle (red)
            2)  # Thickness of the rectangle

    # Display the frame with optical flow vectors
    cv2.imshow('Video', frame)

    # Check if the 'q' key is pressed to quit
    if cv2.waitKey(1) == ord('q'):
        break

    # Update previous image and points for the next iteration
    prevImg = nextImg
    prevPts = nextPts.reshape(-1, 1, 2)

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

### **2.2 Example 2: Multiple Object Tracking w/ Lucas-Kanade**

In [13]:
# Open the video file for reading
cap = cv2.VideoCapture('./data/videos/people.mp4')

# Number of points to track
n = 100

# Size of the window for Lucas-Kanade
winSize = (15, 15)

# Read the first frame from the video
ret, frame = cap.read()

# Convert the first frame to grayscale
prevImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect good features to track using Shi-Tomasi corner detection
points = cv2.goodFeaturesToTrack(prevImg, maxCorners=n, qualityLevel=0.1, minDistance=7)

# Generate random colors for the points
color = np.random.randint(0, 255, (n, 3))

# Store the initial set of points
prevPts = points

# Loop through the frames of the video
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Check if frame is successfully read
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Convert the current frame to grayscale
    nextImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow using Lucas-Kanade method
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, None, winSize=winSize, maxLevel=3)
    
    # Select only the points with valid status
    prevPts = prevPts[status == 1]
    nextPts = nextPts[status == 1]
    points = points[status == 1]
    color = color.reshape(-1, 1, 3)[status == 1]
    
    # Display a line between the old and new positions of the points
    for i, (old_point, new_point) in enumerate(zip(points, nextPts)):
        frame = cv2.line(
            frame, 
            tuple(old_point.astype(int)), 
            tuple(new_point.astype(int)), 
            color[i].tolist(), 
            2)
        
        frame = cv2.rectangle(
            frame, 
            tuple(new_point.astype(int) - np.array(winSize)//2), 
            tuple(new_point.astype(int) + np.array(winSize)//2), 
            color[i].tolist(), 
            2)
    
    # Display the frame with optical flow
    cv2.imshow('frame', frame)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break
    
    # Update the previous frame and points for the next iteration
    prevImg = nextImg
    prevPts = nextPts.reshape(-1, 1, 2)
    points = points.reshape(-1, 1, 2)

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

The code filters out points that were not succesfully tracked and visualizes the movement of the tracked points. It draws lines connecting the old and new positions of the tracked points and draws rectangles around the new positions to indicate their movement.